install libraries

In [10]:
!pip install -q transformers
!pip install -q "datasets<=2.18.0"
!pip install -q sentence-transformers
!pip install -q langchain
!pip install -q langchain_community
!pip install -q torch
!pip install -q numpy
!pip install -q faiss-gpu

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.8 MB/s eta 0:00:00


In [11]:
from datasets import load_dataset

# List available datasets and configurations
imdb_dataset = load_dataset("imdb")

# Print available splits to ensure correct loading
print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [12]:
import re

# Define the word or phrase you want to filter by
specific_word = "Star Wars"

# Define a function to filter texts containing the specific word or phrase
def filter_texts(example):
    return bool(re.search(rf'\b{specific_word}\b', example['text'], re.IGNORECASE))

# Apply the filter to the train and test datasets
filtered_train_dataset = imdb_dataset['train'].filter(filter_texts)
filtered_test_dataset = imdb_dataset['test'].filter(filter_texts)
filtered_unsupervised_dataset = imdb_dataset['unsupervised'].filter(filter_texts)

# Verify the filtered dataset
print(filtered_train_dataset)
print(filtered_test_dataset)
print(filtered_unsupervised_dataset)

print(filtered_train_dataset[0])
print(filtered_test_dataset[0])
print(filtered_unsupervised_dataset[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 159
})
Dataset({
    features: ['text', 'label'],
    num_rows: 153
})
Dataset({
    features: ['text', 'label'],
    num_rows: 306
})
{'text': 'As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one du

In [13]:
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([filtered_train_dataset, filtered_test_dataset, filtered_unsupervised_dataset])

print(combined_dataset)

print(combined_dataset[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 618
})
{'text': 'As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one dude gets the weapon of the aliens and proceeds to use it to go on a very light killing spree. Seriously, you just have to wonder 

In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex=False,
)

# Extract texts from the dataset
texts = combined_dataset["text"]

# Concatenate all texts into a single string
concatenated_text = " ".join(texts)

texts2 = text_splitter.create_documents([concatenated_text])

print(texts2[:2])
# print(texts2[1])


[Document(page_content='As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one dude gets the weapon of the aliens and proceeds to use it to go on a very light killing spree. Seriously, you just have to wonder why this movie was made, if you are going to have 

In [15]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(texts2, embedder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
query = "What is Star Wars?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

be any actual emphasis on plot, it just being an excuse to glue one over-long action set piece to another, so the intrigue disappears fast and the themes and contexts are given a back seat to (admittedly impressive) CGI. It seems that George Lucas thought that what made a good film was as much soulless eye candy as possible.<br /><br />Star wars was once a trailblazer in Science fiction and its originality gave most of the future Blockbusters the templates for their scenes, and what was originally Star Wars innovation became cliché at the hands of other films. Now it is Star Wars using the same plot lines we have seen over and over again, copying others and Episodes 1, 2 and 3 in a lazy attempt to seem interesting.<br /><br />You do not need spectacle to create a good film you need drama and humanity both of which this film sorely lacks. In my opinion the best Light Sabre Fight of the series was the Showdown between Darth Vader and Obi-Wan Kenobi. The fight didn't have any flipping or


In [21]:
# huggingface_embeddings = HuggingFaceBgeEmbeddings(
#     model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
#     model_kwargs={'device':'cpu'},
#     encode_kwargs={'normalize_embeddings': True}
# )

In [20]:
# sample_embedding = np.array(huggingface_embeddings.embed_query(texts2[0].page_content))
# print("Sample embedding of a document chunk: ", sample_embedding)
# print("Size of the embedding: ", sample_embedding.shape)

In [19]:
# vectorstore = FAISS.from_documents(texts2, huggingface_embeddings)

In [17]:
# query = """What is Star Wars."""
#          # Sample question, change to other questions you are interested in.
# relevant_documents = vectorstore.similarity_search(query)
# print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
# print(relevant_documents[0].page_content)